# 15.3预测时间序列

In [1]:
import keras.losses
import numpy as np
def generate_time_series(batch_size, n_steps):
    """
    生成时间序列数据的函数
    参数:
    batch_size (int): 每次生成的序列数量
    n_steps (int): 每个序列的时间步长
    返回:
    numpy.ndarray: 形状为(batch_size, n_steps, 1)的时间序列数据
    """
    # 生成4个随机数数组，分别用于控制两个正弦波的频率和相位偏移
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    # 创建时间序列，从0到1均匀分布n_steps个点
    time = np.linspace(0, 1, n_steps)
    # 生成第一个正弦波，振幅为0.5
    # 通过随机频率(freq1*10+10)和相位偏移(offsets1)控制波形
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  # wave 1
    # 添加第二个正弦波，振幅为0.2
    # 通过不同的随机频率(freq2*20+20)和相位偏移(offsets2)控制波形
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20))  # wave 2
    # 添加随机噪声，振幅为0.1
    # 使用均匀分布生成随机数并减去0.5使其均值为0
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)  # noise
    # 调整数组形状并转换为float32类型
    return series[..., np.newaxis].astype(np.float32)


2025-10-30 17:52:20.704235: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-30 17:52:20.710496: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-30 17:52:20.719098: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-30 17:52:20.721547: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-30 17:52:20.727634: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# 创建训练集 验证集 测试集
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train,y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

### 15.3.1 基准指标

In [4]:
from tensorflow import keras

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50, 1]),
    keras.layers.Dense(1)
])
model.compile(loss='mse', optimizer='adam')
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))
model.evaluate(X_test, y_test)

Epoch 1/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0792 - val_loss: 0.0462
Epoch 2/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 872us/step - loss: 0.0336 - val_loss: 0.0253
Epoch 3/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 960us/step - loss: 0.0199 - val_loss: 0.0163
Epoch 4/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 875us/step - loss: 0.0141 - val_loss: 0.0122
Epoch 5/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 959us/step - loss: 0.0112 - val_loss: 0.0101
Epoch 6/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 930us/step - loss: 0.0095 - val_loss: 0.0087
Epoch 7/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 859us/step - loss: 0.0083 - val_loss: 0.0077
Epoch 8/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 840us/step - loss: 0.0073 - val_loss: 0.0068
Epoch 9/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0065 - val_loss: 0.0061
Epoch 10/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 905us/step - loss: 0.0059 - val_loss: 0.0056
Epoch 11/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 878us/step - loss: 0.0055 - val_loss: 0.0052
Epoch 12/20
219/219 ━━━━━━━━━━━━━━

0.00349472020752728

### 15.3.2 使用RNN

In [8]:
from tensorflow import keras

# 使用RNN
# 目标：预测序列的下一个时间步（单值），即 Sequence-to-One 任务

model = keras.models.Sequential([
    # 1. SimpleRNN：
    #    - 移除 return_sequences=True：让 RNN 只输出最后一个时间步的隐藏状态。
    #    - 此时输出形状变为 (None, 50)
    keras.layers.SimpleRNN(50, input_shape=[n_steps, 1]),

    # 2. Dense 层：
    #    - 添加一个 Dense(1) 层，将 50 个神经元压缩为最终的 1 个预测值。
    #    - 最终模型输出形状变为 (None, 1)，与 y_train 的形状 (None, 1) 兼容。
    keras.layers.Dense(1)
])

model.compile(loss='mse', optimizer='adam')
# 现在可以正确运行了
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))
model.evaluate(X_test, y_test)

Epoch 1/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0143 - val_loss: 0.0044
Epoch 2/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0039 - val_loss: 0.0035
Epoch 3/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0035 - val_loss: 0.0031
Epoch 4/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0032 - val_loss: 0.0032
Epoch 5/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0032 - val_loss: 0.0034
Epoch 6/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0031 - val_loss: 0.0029
Epoch 7/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0031 - val_loss: 0.0031
Epoch 8/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0030 - val_loss: 0.0027
Epoch 9/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0030 - val_loss: 0.0030
Epoch 10/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0030 - val_loss: 0.0027
Epoch 11/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0030 - val_loss: 0.0027
Epoch 12/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step

0.002343049505725503